Chapter 14: Voxels
==================

Voxel plots
-----------

A voxel is the 3D analogue of the pixel. This type of visualization,
completely new in gnuplot v. 5.4, allows the rendering of data in 3D
space. Voxel data are familiar in medical imaging, where they are used
to display the results of MRIs or CAT scans, and in engineering or
physics, where they are helpful in understanding such things as the 3D
flow pattern around a propeller. Up to now, “3D” plotting in gnuplot was
confined to rendering surfaces in various ways, drawing curves, or
placing points within the 3D space: in other words, 2D, 1D, or 0D
objects embedded in a volume region. All these types of plots are
covered in previous chapters. Voxel plotting, in contrast, is true 3D
visualization of 3D information; you can think of it as the 3D version
of a 2D heatmap. What this means will become clear with the examples in
this chapter.

Another way to understand this is that a surface plot is a visualization
of a function of two variables: $z = f(x, y)$. With voxel plotting, we
can now visualize functions of three variables: $f(x, y, z)$.

All voxel plotting begins with the establishment of a voxel grid, or
`vgrid`. This is simply a regular cubic array of places in which to
store values. It is perhaps better to think of the volume as divided
into an array of subvolumes, each of which is filled with its local
value, than as divided into an array of points. The command to set up a
100 × 100 × 100 grid, and give it the name `$v`, is
`set vgrid $v size 100` (the names of voxel grids must start with a
dollar sign). A new voxel grid is initialized with 0 at every point.

If you want to reinitialize the vgrid after working with it, you must do
`unset vgrid $v`, using whatever name you have assigned it, and then
enter `set vgrid` again; simply reentering `set vgrid` with the same
dimensions will not zero the voxel values. Another option is `vclear`,
which zeroes the active grid, or a different one with an argument.

Before we do any plotting with this grid, we want, as usual, to set the
ranges on the axes; but now, there are two sets of ranges: the usual
`xrange`, etc., for the box that holds the plot, and new ones called
`vxrange`, etc, for the coordinate extents of the voxel grid. The voxel
grid has an existence independent of the grids used to plot it, or the
coordinate system in which it is placed.

Now that we have our voxel grid, we need to fill it with values in order
to have something to plot. It will be instructive to use one example,
with a simple physical interpretation, and show how to visualize it in
several ways. Our example for the rest of this chapter will be the
potential field (“voltage”) surrounding two equal and opposite charges:
an electric dipole. This is a model, for example, of the [potential
field around a water
molecule](http://www1.lsbu.ac.uk/water/water_molecule.html), if you are
not too close to the molecule.

One of the capabilities recently added to gnuplot is the ability to
store blocks of data in named variables. We’ll use this to define a
little data block that holds the locations of the two charges and their
charge values. If we use a range of `[0 : 1]`, the command to create the
data block can be

    $charges << EOD
    0 0 0.75 1
    0 0 0.25 -1
    EOD

In `$charges`, the first three columns are x, y, and z, so the charges
are on the z-axis. The fourth column is for the charge values.

After this, we can use `$charges` in any command where we need to refer
to this data. Just by changing this command, you can alter all the
examples in this chapter to plot the potential field surrounding any
configuration of charges that you want.

Now we need to do something with these coordinates, and that brings us
to our second new command, `vfill`. This command is a little confusing
to wrap your head around because it operates on two grids at once. It
takes a grid of coordinates, which need not be the same size as the
voxel grid, and, for each point therein, finds all the points in the
voxel grid that lie within a given radius of the point. Each of these
voxel locations gets a specified value added to it.

Proceeding with our example may make this more concrete. We’re going to
use the `vfill` command to add the contribution to the potential field
from each charge to the voxel grid. For this purpose, it will be
convenient to have a function that calculates that potential at a given
distance from the charge:

    pot(r) = r > 0 ? 1/r : 10^6                                                                              

Here we have used gnuplot’s ternary notation, which borrows from c and
other languages. The function `pot` will return the usual potential for
positive distances, but gives us an arbitrary large number if we are
right on top of the charge, where the potential would be infinite.

We can use this in a `vfill` command to populate the voxel grid this
way:

    vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))

This command will consider each charge in turn. For each charge, it will
determine which voxels are within a radius of 2 of the charge, and add
to the value there the value of the potential. Gnuplot supplies the
convenience function `VoxelDistance`, that returns the distance from
each point in the voxel grid to the grid point under consideration. For
the radius, we just wanted to include every point in the grid, so we
used a value larger than the largest distance within the cube
($\sqrt{3}$); anything larger would have the same effect. This command,
and all the plotting commands below that will refer to voxel values, use
the *currently active voxel grid*, which is the one most recently
defined. As you can see, we’re not bothering with physical constants or
units; we are only interested in the shape of the potential field. If we
want to, we can convert to physical units at any time with a simple
scaling.

To summarize, the command in general takes the form

    vfill XYX-coordinates using x:y:z:(radius):(value)

Now that we have populated the voxel grid with values, we would
naturally like to look at them, to see the configuration of the
potential field. Gnuplot gives us four distinct ways to do this, and
they are all variations of the familiar `splot` command for plotting
surfaces. The examples in the rest of this chapter will show us how to
use all of these new plot commands.

There is a new function, `voxel(x, y, z)` which returns the value stored
in the active voxel grid at the specified 3D location. \\index{voxel
function Note that the current release (version 5.4 patchlevel rc2) will
crash with a segfault if you ask for `voxel(x, y, z)` after initalizing
the voxel grid but before adding any values with `vfill`.

Plotting Points in 3D
---------------------

The `splot` command can now plot a collection of points in 3D with their
colors, sizes, and types taken from any data source, as before in 2D;
the voxel grid is simply an additional source of data. So we can begin
by making a picture showing the location of our charges, without
reference to the voxel grid at all. These examples, as in every chapter,
are all self-contained: you can run these scripts as-is in gnuplot (as
long as you are using v. 5.4) and they will produce the plot shown. That
is the reason for the repeated setup in many of the examples.

In [ ]:
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set samp 100,100; set iso 100,100
set view 65,40
set xyplane at -0.1
set border 4095
unset key
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
splot $charges using 1:2:3 with points pt 7 ps 5 lc pal

Once a voxel grid is defined and made active with the `set vgrid`
command, and filled with values using the `vfill` command, you can
visualize it in several ways. The first method we’re going to look at is
a simple variation of the `splot` command that draws a point in 3D space
for each point in the voxel grid. You can color these points with a
constant color or with a palette. This command takes a clause `above x`;
it will plot points only for voxel values greater than `x`. The default
value is 0, so if this is omitted you will get points only where the
voxels are positive. Since you are limited to constant or palette
colors, and the command does not accept a `using` clause, this form of
the `splot` command is limited. You can not have variable colors with
transparency, nor can you make the point sizes or types depend on the
data; but we will see other ways to do all these things later in the
chapter.

Here we use two `splot...above` commands to draw two nested volumes
showing the configuration of the potential near the negative charge.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
dx = 1.0/20
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set xyplane at -0.1; set border 4095; unset key
splot $v with points above 5 pt 7 ps 0.2,\
    $v with points above 10 pt 7 ps 0.2 lc "green"

This invocation of the `splot` command accepts a `pointinterval`
setting, in case you need a less dense sampling of the voxel grid.
Plotting fewer points helps us to see around them and get a better sense
of the interior structure of the field; this can be helped by
interactively rotating the plot if you are using a terminal, such as
x11, that supports that.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
dx = 1.0/20
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set xyplane at -0.1; set border 4095; unset key
splot $v with points above 5 pt 7 ps 0.2 pi 4,\
    $v with points above 10 pt 7 ps 0.3 pi 4 lc "green"

Here we color the points using the default palette, with a
`pointinterval` setting to let us try to see inside. We use a small
`above` setting to image most of the voxel grid.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set cbr [-3 : 3]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set xyplane at -0.1; set border 4095; unset key
splot $v with points above -100 pt 7 ps 0.2 pi 4 lc pal

<div class = "chintro">

Jitter
------

You may have noticed the appearance of a kind of moiré pattern in the
previous visualizations. This is due to viewing periodic arrays of
points superimposed on each other, and will be a problem in many of
these 3D plots using arrays of points.

The moiré pattern that results from looking through a regular array of
points is not only annoying, but can give a false impression about the
nature of the data that the visualization is trying to clarify. You can
eliminate this problem by applying `jitter`.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set cbr [-3 : 3]
set xyplane at -0.1; set border 4095; unset key
set jitter over 0 spread  3 square
splot $v with points above -100 pt 7 ps 0.2 pi 4 lc pal

If you don’t care about seeing inside the box, but are content to admire
surface appearances, you can just use a larger point size.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
dx = 1.0/20
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set cbr [-3 : 3]
set xyplane at -0.1; set border 4095; unset key
set jitter over 0 spread  3 square
splot $v with points above -100 pt 7 ps 0.4 lc pal

Creating Coordinate Files
-------------------------

In order to have more options when making the type of volume plot that
we saw in the last few examples, we will have to make a coordinate grid
and `splot` it, using the voxel grid as a data source. This will allow
us to include a `using` clause, which means that we’ll be able to vary
the opacity, color, point size, and point type with the voxel data. The
next few examples will explore some of the possibilities.

Although our examples are self-contained, the ones that follow will
require a data file consisting of a grid of coordinates. In 1D and 2D
`splot` commands, you can use the “special filenames” `+` and `++` to
automatically generate these grids on the fly, and refer to their
columns in a `using` clause. Because gnuplot does not have a `+++`
special filename, we’ll have to generate these coordinates in a separate
step and store them on disk. We could also store them in a datablock, as
we did to define the locations of the charges above, but, since we will
be using these coordinates repeatedly, it’s more convenient to do it
once. In order for any of the following examples that refer to `cube20`
or `cube100` to work, you must have run the following code in gnuplot,
to generate the files. We’ll use gnuplot’s convenient looping commands,
and the `set print` command to print the output to a file:

    nx = 20
    set print "cube" . nx
    do for [i = 0 : nx] {
      do for [j = 0 : nx] {
        do for [k = 0 : nx] {
          ic = 1.0*i/nx
          jc = 1.0*j/nx
          kc = 1.0*k/nx
          print sprintf("%f %f %f", ic, jc, kc)
        }}}

You should run the above code twice; once as is, and once after doing
`nx = 100` to produce the `grid100` file. Make sure the you run the
examples in the same directory containing these grid files, or they
won’t work as shown.

Volume Plot from a Voxel Grid
-----------------------------

Here we show how to plot a projection, or sampling, of the voxel grid on
a set of 3D coordinates. This type of plot is like the familiar
scatterplot in 2D, that we create with the `plot...with points` command,
but now the plotted points will be arranged within the 3D volume region.
We use a macro for the `voxel` function to make our `splot` command more
concise. Since we would like to clearly distinguish between negative and
positive values of the potential, we’ll define a palette that shows
this.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set pal define (0 "red", .5 "black", 1 "blue")
v = "voxel($1, $2, $3)"
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set view 65, 40; set xyplane at -0.1; set border 4095
unset key; set cbr [-4 : 4]
splot "cube100" using 1:2:3:(@v) with points pt 7 ps .4 lc pal

This is pretty nice, but notice that, as in a previous example, because
you can’t see through the points on the surfaces of the cube, you can’t
see inside it; so you’re really just looking at several 2D surface
plots. We could use fewer points, to allow us to see around them. This
will show us some of what’s going on inside. We’ll make one change to
the previous script, using the 20³ grid rather than the 100³ one. In
other words, instead of skipping points by setting a `pointinterval`,
we’ll use the sparser grid that we saved on disk. In fact, the
`pointinterval` command does not work with this version of the `splot`
command.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set pal define (0 "red", .5 "black", 1 "blue")
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
v = "voxel($1, $2, $3)"
set view 65, 40
set xyplane at -0.1
set border 4095
unset key
set cbr [-4 : 4]
splot "cube20" using 1:2:3:(@v) with points pt 7 ps .4 lc pal

Manual Jitter
-------------

You may have noticed the reappearance of the moiré pattern in this
visualization. The `jitter` setting has no effect when using this
version of the `splot` command, so we’ll have to roll our own. As a
consolation, doing it ourselves will allow us to exercise more control.

We’ll begin by defining a macro that we can use in plot commands to add
some random variation to the coordinates:

    j = '(rand(0) - 0.5) * dx'

To use this, we should define `dx` to be the distance between
neighboring points on the grid. Now all we need to do is to add this to
each coordinate, as in the following examples.

Voxel Plot with Jitter
----------------------

Here we repeat the previous plot, but using our home-made jitter macro.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
dx = 1.0/20
set pal define (0 "red", .5 "black", 1 "blue")
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
j = "(rand(0) - 0.5) * dx"
v = "voxel($1, $2, $3)"
set view 65, 40
set xyplane at -0.1
set border 4095
unset key
set cbr [-4 : 4]; unset colorbox
splot "cube20" using ($1 + @j):($2 + @j):($3 + @j):(@v) with points pt 7 ps .4 lc pal

We might be able to improve the resolution a bit by using more points,
while using `dots`, which will draw the smallest possible point. We’ll
also tilt the box a bit more.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
dx = 1.0/20
set pal define (0 "red", .5 "black", 1 "blue")
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
j = "(rand(0) - 0.5) * dx"
v = "voxel($1, $2, $3)"
set view 50, 40
set xyplane at -0.1
set border 4095
unset key
set cbr [-4 : 4]; unset colorbox
splot "cube100" using ($1 + @j):($2 + @j):($3 + @j):(@v)\
    with dots lc pal

As you can see, even if we make the points as small as possible, if we
try to get good resolution by using a fine grid, we can’t see inside
very far, as the points occlude each other. One obvious thing to try is
to use transparent colors for the points, so we can see through them.
Since palettes in gnuplot can’t have transparency, we’ll have to specify
the colors some other way. For this, it will be convenient to define
some functions that map field values into color numbers.

First we store the minimum and maximum of the data in `dmin` and `dmax`.
We will be working with the integer representation of colors in gnuplot,
which are most conveniently expressed as a hexadecimal number with the
format `0xAARRGGBB`, where the high bits, the `A`s, represent the
opacity, which goes from completely opaque, at `0x00RRGGBB`, to
completely transparent, or invisible, at `0xFFRRGGBB`. The other places
represent the values for red, green and blue. This representation makes
adding or subtracting particular color values, or opacity, convenient,
if we use bit shifts. As an example, if we start with black =
`0x00000000` and want to turn it into maximum green, we can add to it
the number `255<<8 = 65280 = 0xFF00 = 255 * 2**8`. Any of these
representations will work, but the bit shift operator, the first one, is
the most concise way to convert values from 0 to 255 into numbers for
the colors that we want. Using this technique, we can make a function
that maps \[0, 1\] to \[black, blue\] like this:

    bb(x, dmin, dmax) = int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF)

and one that goes from black to red like this:

    br(x, dmin, dmax) = (int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF))<<16

(The multiplications by 1.0 are to force a conversion to float, because
1/2 = 0 in gnuplot. Also, since gnuplot doesn’t come with a min nor a
max function, we’ll have to make our own.)

What we are really after is a function that goes from red in the
negative to blue in the positive, with black near zero, similar to the
palette we’ve been using up to now. We can put this together using the
two functions we just defined:

    pmrb(x, dmin, dmax) = x<0?br(abs(x), 0, abs(dmin)):bb(x, 0, dmax)

Once we have this color function, we can make the colors transparent by
adding a number like 0xFB000000 to them; this particular choice will
make them nearly totally transparent, but since there are so many
overlapping points, and the opacity builds up with the overlap, this is
what we need. The next example shows the result.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
max(a, b) = a<b?b:a
min(a, b) = a<b?a:b
bb(x, dmin, dmax) = int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF)
br(x, dmin, dmax) = (int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF))<<16
pmrb(x, dmin, dmax) = x<0?br(abs(x), 0, abs(dmin)):bb(x, 0, dmax)
dx = 1.0/100
j = "(rand(0) - 0.5) * dx"; v = "voxel($1, $2, $3)"
set view 50, 40; set xyplane at -0.1; set border 4095
unset key
splot "cube100" using ($1 + @j):($2 + @j):($3 + @j):(pmrb(@v, -2, 2) +\
    0xFB000000) with points pt 7 ps .2 lc rgbcolor var

We can create a somewhat more refined visualization by varying the
opacity with the field strength. In this script you will see another
function, `tb`, that maps field values to opacity values, by shifting a
number in \[0, 255\] to the high bits of the `rgbcolor` integer.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
max(a, b) = a<b?b:a; min(a, b) = a<b?a:b
bb(x, dmin, dmax) = int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF)
br(x, dmin, dmax) = (int(1.0*(min(max(x, dmin), dmax))/dmax*0xFF))<<16
pmrb(x, dmin, dmax) = x<0?br(abs(x), 0, abs(dmin)):bb(x, 0, dmax)
tb(x, dmax) = -((int(1.0*min(abs(x), abs(dmax))/abs(dmax)*0xFF))<<24)
pot(r) = r > 0 ? 1/r : 10^6
dx = 1.0/100
j = "(rand(0) - 0.5) * dx"; v = "voxel($1, $2, $3)"
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set view 65, 100; set xyplane at -0.1; set border 4095; unset key
splot "cube100" using ($1 + @j):($2 + @j):($3 + @j):\
    (0xFF000000 + pmrb(@v, -5, 5) + tb(@v, 20))\
    with points pt 7 ps .2 lc rgbcolor var

Up to now we have used the voxel values to set the color or opacity of
the gridpoints. As with 2D `splot`s, we can also scale the point size
using the data. There are other possibilities, of course, such as
combining this technique with variable opacity, but after this example
it will be time to change gears.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set pal define (0 "red", .5 "black", 1 "blue"); set cbr [-1 : 1]
dx = 1.0/100
j = "(rand(0) - 0.5) * dx"; v = "voxel($1, $2, $3)"
min(a, b) = a<b?a:b
set view 65, 100; set xyplane at -0.1; set border 4095; unset key
splot "cube20" using\
   ($1 + @j):($2 + @j):($3 + @j):(min(abs(@v)*.2, 3)):(voxel($1,$2,$3))\
   with points pt 7 ps var lc pal

Another common way to visualize the interior of a volume is to intersect
it with a plane and plot the projection of the field on the plane
surface. We can project the voxel grid on a plane in any orientation and
at any position. We continue to jitter the coordinates in all the
similar examples in this chapter.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
dx = 1.0/100
set samp 100; set iso 100
j = "(rand(0) - 0.5) * dx"; v = "voxel($1, $2, $3)"
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pal define (0 "red", .5 "black", 1 "blue"); set cbr [-1 : 1]
splot "cube20" using ($1 + @j):($2 + @j):($3 + @j):(voxel($1,$2,$3))\
    with points pt 7 ps .3 lc pal,\
    "++" using (0.2):1:2:(voxel(0.2, $1, $2)) with pm3d

We can extend the previous method to include a series of slices, which
is one of the most effective ways to visualize a 3D field. We’ll have to
make the slices transparent for this to work. We’ll also change the
palette to use white for the field values near zero. The final command
uses [a loop](#basic-iteration) to plot the slices; we’re leaving out
the plotting of points here, so we no longer need the jitter function.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set samp 100; set iso 100
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pal define (0 "red", .5 "white", 1 "blue"); set cbr [-1 : 1]
set style fill transparent solid 0.4
splot for [j=0:9] "++" using (j/10.):1:2:(voxel(j/10., $1, $2))\
    with pm3d

We need not be confined to slicing the data with planes parallel to a
coordinate plane, as in the previous example. Here we slice with a plane
going diagonally across the box.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set samp 100; set iso 100
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
dx = 1.0/20; j = "(rand(0) - 0.5) * dx"; v = "voxel($1, $2, $3)"
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pal define (0 "red", .5 "black", 1 "blue"); set cbr [-0.5 : 0.5]
splot "cube20" using ($1 + @j):($2 + @j):($3 + @j):(voxel($1,$2,$3))\
    with points pt 7 ps 0.3 lc pal,\
    "++" using (1 - $1):1:2:(voxel(1 - $1, $1, $2)) with pm3d

Our intersecting surface need not even be a plane. We can image the
projection of the voxel grid on a curved surface, as well. This example
bends the plane in the previous example into a section of a circle. The
`depthorder` setting is to cause the surface to render correctly in 3D
perspective.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set samp 100; set iso 100; v = "voxel($1, $2, $3)"
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
dx = 1.0/20; j = "(rand(0) - 0.5) * dx"
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pal define (0 "red", .5 "black", 1 "blue"); set cbr [-0.2 : 0.2]
set pm3d depthorder
splot "cube20" using\
  ($1 + @j):($2 + @j):($3 + @j):(voxel($1,$2,$3))\
  with points pt 7 ps .3 lc pal,\
   "++" using (cos($1*pi/2.0)):(sin($1*pi/2.0)):2:(voxel(cos($1*pi/2.0), sin($1*pi/2.0), $2)) with pm3d

Of course, you can also sample the voxel values with a curve passing
through the 3D space.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set pal define (0 "red", .5 "black", 1 "blue"); set cbr [-0.2 : 0.2]
set pm3d depthorder
splot "cube100" using 1:1:(($1)**0.5):(voxel($1, $1, ($1)**0.5)) with points pt 7 ps 1 lc pal

The `splot` command for visualizing voxel grids has one more trick up
its sleeve: isosurfaces. This is a surface, or a set of surfaces, that
show where the voxel grid has a certain value (interpolating if
necessary). The following command draws the isosurfaces showing where
the voxel grid has the values 1 or -1. Notice how the “top” and “bottom”
of the surfaces are colored differently, and how we don’t need to supply
a coordinate grid. This command uses `pm3d` surfaces behind the scenes;
hence the `depthorder` command to make them render correctly.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pm3d depthorder
splot $v with isosurface level 1, $v with isosurface level -1

We can produce an excellent visualization of this field by drawing a set
of nested isosurfaces, looping as we did above when we plotted a series
of slices. This graph makes the symmetry of the field clear, while also
giving a good indication of how intensity falls off with distance from
the charges. Unfortunately, in the current version (v. 5.4 patchlevel
rc2), the coloring of isosurfaces is broken, so for now we need to live
with the sequence of colors that we get.

In [ ]:
set vgrid $v size 100
set vzrange [0:1]; set vyrange [0:1]; set vzrange [0:1]
set urange [0:1]; set vrange [0:1]
set xrange [0:1]; set yrange [0:1]; set zrange [0:1]
$charges << EOD
0 0 0.75 1
0 0 0.25 -1
EOD
pot(r) = r > 0 ? 1/r : 10^6
vfill $charges using 1:2:3:(2):($4*pot(VoxelDistance))
set view 65, 20; set xyplane at -0.1; set border 4095; unset key
set pm3d depthorder
set style fill transparent solid 0.4
splot for [j=-100:100:1] $v with isosurface level j